In [65]:
import torch

In [2]:
import torch
from sympy.printing.tensorflow import tensorflow

In [2]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from torch_geometric.data import Data

C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [91]:
data=torch.load('graph_data_node2vec_weights_and_text.pt')

C:\Users\Dell\AppData\Local\Temp\ipykernel_34480\12758277.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data=torch.load('graph_data_node2vec_weights_and_text.pt')


In [92]:
#data.y[data.y == -1] = 3

In [93]:
# num_nodes = data.num_nodes
# num_train = int(0.8 * num_nodes)
# train_mask = torch.zeros(num_nodes, dtype=torch.bool)
# train_mask[:num_train] = 1

In [94]:
# data.train_mask = train_mask
# data.test_mask = ~train_mask

In [95]:
num_nodes = data.num_nodes
train_ratio, val_ratio = 0.8, 0.1

perm = torch.randperm(num_nodes)
train_size = int(train_ratio * num_nodes)
val_size = int(val_ratio * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [96]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv

In [97]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Dropout, LeakyReLU


In [98]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [99]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import SAGEConv
class SAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(-1, 64)
        self.conv2 = SAGEConv(64, 128)
        self.conv3 = SAGEConv(128, 256)
        self.conv4 = SAGEConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [100]:
# def train(model, dataset, optimizer, criterion, epochs=5):
#     train_mask = dataset.train_mask
#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         out = model(dataset.x, dataset.edge_index,dataset.edge_weight).squeeze()  # Output shape: [num_nodes]
#         loss = criterion(out[train_mask], dataset.y[train_mask].float()) 
#         loss.backward()
#         optimizer.step()
#         print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [101]:
def train(model, dataset, optimizer, criterion, epochs=5):
    train_mask = dataset.train_mask
    val_mask = dataset.val_mask  # Assuming `val_mask` is available in the dataset
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        out = model(dataset.x, dataset.edge_index, dataset.edge_weight).squeeze()
        
        # Compute loss on the training set
        loss = criterion(out[train_mask], dataset.y[train_mask].float())
        loss.backward()
        optimizer.step()

        # Validation Performance
        model.eval()
        with torch.no_grad():
            val_out = out[val_mask]
            val_loss = criterion(val_out, dataset.y[val_mask].float())
            val_pred = (torch.sigmoid(val_out) > 0.5).long()
            # val_acc = accuracy_score(dataset.y[val_mask].cpu(), val_pred.cpu())
        
        print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

In [102]:
data

Data(x=[4150, 66], edge_index=[2, 439838], y=[4150], edge_weights=[439838], train_mask=[4150], val_mask=[4150], test_mask=[4150])

In [103]:
model_gcn = GCN()

In [104]:
model_sage=SAGE()

In [105]:
model_gcn

GCN(
  (conv1): GCNConv(-1, 64)
  (conv2): GCNConv(64, 128)
  (conv3): GCNConv(128, 256)
  (conv4): GCNConv(256, 512)
  (linear): Linear(in_features=512, out_features=256, bias=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [106]:
optimizer = Adam(model_gcn.parameters(), lr=0.001)

In [107]:
criterion = torch.nn.BCEWithLogitsLoss()

In [62]:
(data.y == 0).sum().item()

3064

In [41]:
train(model=model_sage, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

TypeError: SAGE.forward() takes 3 positional arguments but 4 were given

In [108]:
train(model=model_gcn, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

Epoch: 000, Train Loss: 0.7202, Val Loss: 0.7231
Epoch: 001, Train Loss: 0.6586, Val Loss: 0.6535
Epoch: 002, Train Loss: 0.6011, Val Loss: 0.5789
Epoch: 003, Train Loss: 0.5932, Val Loss: 0.5420
Epoch: 004, Train Loss: 0.6138, Val Loss: 0.5521
Epoch: 005, Train Loss: 0.5881, Val Loss: 0.5382
Epoch: 006, Train Loss: 0.5779, Val Loss: 0.5471
Epoch: 007, Train Loss: 0.5828, Val Loss: 0.5548
Epoch: 008, Train Loss: 0.5872, Val Loss: 0.5651
Epoch: 009, Train Loss: 0.5842, Val Loss: 0.5617
Epoch: 010, Train Loss: 0.5752, Val Loss: 0.5507
Epoch: 011, Train Loss: 0.5703, Val Loss: 0.5332
Epoch: 012, Train Loss: 0.5679, Val Loss: 0.5273
Epoch: 013, Train Loss: 0.5682, Val Loss: 0.5261
Epoch: 014, Train Loss: 0.5818, Val Loss: 0.5290
Epoch: 015, Train Loss: 0.5619, Val Loss: 0.5223
Epoch: 016, Train Loss: 0.5605, Val Loss: 0.5259
Epoch: 017, Train Loss: 0.5645, Val Loss: 0.5416
Epoch: 018, Train Loss: 0.5691, Val Loss: 0.5486
Epoch: 019, Train Loss: 0.5608, Val Loss: 0.5354
Epoch: 020, Train Lo

In [88]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [109]:
def test(model, dataset):
    model.eval()
    test_mask = dataset.test_mask

    with torch.no_grad():
        out = model(dataset.x, dataset.edge_index, dataset.edge_weight).squeeze()
        test_out = out[test_mask]
        test_pred = (torch.sigmoid(test_out) > 0.5).long()
        test_labels = dataset.y[test_mask]

    # Calculate metrics
    test_acc = accuracy_score(test_labels.cpu(), test_pred.cpu())
    test_precision = precision_score(test_labels.cpu(), test_pred.cpu(), average='macro')
    test_recall = recall_score(test_labels.cpu(), test_pred.cpu(), average='macro')
    test_f1 = f1_score(test_labels.cpu(), test_pred.cpu(), average='macro')

    print(f'Test Accuracy: {test_acc:}')
    print(f'Test Precision: {test_precision:}')
    print(f'Test Recall: {test_recall:}')
    print(f'Test F1 Score: {test_f1:}')

In [110]:
test(model=model_gcn, dataset=data)

Test Accuracy: 0.7734939759036145
Test Precision: 0.7215203127181349
Test Recall: 0.5968889295963852
Test F1 Score: 0.6050979915775834


With only node2vec: <br/>
Test Accuracy: 0.7542168674698795 <br/>
Test Precision: 0.6575286415711947 <br/>
Test Recall: 0.5709677419354838 <br/>
Test F1 Score: 0.5714893100097181 <br/>
<br/>

With node2vec+ graph weights(how many times the users have commented on one another) : <br/>
Test Accuracy: 0.7614457831325301 <br/>
Test Precision: 0.6804091477577273 <br/>
Test Recall: 0.5624381647291615 <br/>
Test F1 Score: 0.5568008975091963 <br/>
<br/>

With node2vec + graph weights + text_embeddings: <br/>
Test Accuracy: 0.7734939759036145 <br/>
Test Precision: 0.7215203127181349 <br/>
Test Recall: 0.5968889295963852 <br/>
Test F1 Score: 0.6050979915775834 <br/>




In [178]:
from sklearn.cluster import KMeans

In [27]:
from sklearn.cluster import AgglomerativeClustering

# USING Class Weighting

In [81]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [82]:
labels = data.y.cpu().numpy() 
classes = np.unique(labels)

In [83]:
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)

In [84]:
weights = torch.tensor(class_weights, dtype=torch.float) 

In [85]:
criterion2 = torch.nn.BCEWithLogitsLoss(pos_weight=weights[1])


In [86]:
model_gcn2 = GCN()
optimizer = Adam(model_gcn.parameters(), lr=0.001)

In [87]:
train(model=model_gcn2, dataset=data,
      criterion=criterion2, optimizer=optimizer,
      epochs=50)

Epoch: 000, Train Loss: 0.8680, Val Loss: 0.8647
Epoch: 001, Train Loss: 0.8685, Val Loss: 0.8664
Epoch: 002, Train Loss: 0.8687, Val Loss: 0.8656
Epoch: 003, Train Loss: 0.8686, Val Loss: 0.8658
Epoch: 004, Train Loss: 0.8693, Val Loss: 0.8665
Epoch: 005, Train Loss: 0.8680, Val Loss: 0.8652
Epoch: 006, Train Loss: 0.8688, Val Loss: 0.8658
Epoch: 007, Train Loss: 0.8696, Val Loss: 0.8664
Epoch: 008, Train Loss: 0.8678, Val Loss: 0.8656
Epoch: 009, Train Loss: 0.8687, Val Loss: 0.8655
Epoch: 010, Train Loss: 0.8693, Val Loss: 0.8672
Epoch: 011, Train Loss: 0.8679, Val Loss: 0.8654
Epoch: 012, Train Loss: 0.8685, Val Loss: 0.8652
Epoch: 013, Train Loss: 0.8686, Val Loss: 0.8650
Epoch: 014, Train Loss: 0.8688, Val Loss: 0.8673
Epoch: 015, Train Loss: 0.8689, Val Loss: 0.8661
Epoch: 016, Train Loss: 0.8685, Val Loss: 0.8658
Epoch: 017, Train Loss: 0.8688, Val Loss: 0.8654
Epoch: 018, Train Loss: 0.8682, Val Loss: 0.8656
Epoch: 019, Train Loss: 0.8687, Val Loss: 0.8654
Epoch: 020, Train Lo

In [88]:
test(model=model_gcn2, dataset=data)

Test Accuracy: 0.26987951807228916
Test Precision: 0.13493975903614458
Test Recall: 0.5
Test F1 Score: 0.2125237191650854


C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## FAKE WEIGHTS

In [87]:
weights[1]

tensor(1.9107)

In [88]:
fake_weight=torch.tensor(2.5)

In [89]:
criterion3 = torch.nn.BCEWithLogitsLoss(pos_weight=fake_weight)


In [90]:
train(model=model_gcn, dataset=data,
      criterion=criterion3, optimizer=optimizer,
      epochs=50)

Epoch: 000, Loss: 0.8760
Epoch: 001, Loss: 0.8475
Epoch: 002, Loss: 0.8552
Epoch: 003, Loss: 0.8509
Epoch: 004, Loss: 0.8515
Epoch: 005, Loss: 0.8597
Epoch: 006, Loss: 0.8483
Epoch: 007, Loss: 0.8585
Epoch: 008, Loss: 0.8648
Epoch: 009, Loss: 0.8415
Epoch: 010, Loss: 0.8422
Epoch: 011, Loss: 0.8548
Epoch: 012, Loss: 0.8448
Epoch: 013, Loss: 0.8604
Epoch: 014, Loss: 0.8491
Epoch: 015, Loss: 0.8452
Epoch: 016, Loss: 0.8487
Epoch: 017, Loss: 0.8586
Epoch: 018, Loss: 0.8624
Epoch: 019, Loss: 0.8445
Epoch: 020, Loss: 0.8388
Epoch: 021, Loss: 0.8555
Epoch: 022, Loss: 0.8540
Epoch: 023, Loss: 0.8532
Epoch: 024, Loss: 0.8429
Epoch: 025, Loss: 0.8584
Epoch: 026, Loss: 0.8514
Epoch: 027, Loss: 0.8592
Epoch: 028, Loss: 0.8484
Epoch: 029, Loss: 0.8436
Epoch: 030, Loss: 0.8495
Epoch: 031, Loss: 0.8462
Epoch: 032, Loss: 0.8428
Epoch: 033, Loss: 0.8470
Epoch: 034, Loss: 0.8442
Epoch: 035, Loss: 0.8506
Epoch: 036, Loss: 0.8413
Epoch: 037, Loss: 0.8414
Epoch: 038, Loss: 0.8412
Epoch: 039, Loss: 0.8415


In [50]:
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import networkx as nx

In [57]:
from torch_geometric.utils import to_networkx

# Convert to a NetworkX graph
# Assuming your `data` object contains `edge_index` and `x` (node features)
G = to_networkx(data, to_undirected=True)  # Convert to an undirected NetworkX graph

# Optionally, add node features or labels as attributes to the NetworkX graph
for i, feature in enumerate(data.x):
    G.nodes[i]['feature'] = feature.tolist()
    
if hasattr(data, 'y'):  # Add labels if they exist
    for i, label in enumerate(data.y):
        G.nodes[i]['label'] = int(label.item())


In [58]:
for u, v, idx in G.edges(data=True):
    idx['weight'] = float(data.edge_weight[data.edge_index[0] == u][data.edge_index[1] == v])

TypeError: 'NoneType' object is not subscriptable

In [55]:
export_to_gephi(data)

TypeError: cannot unpack non-iterable float object

In [53]:
G = to_networkx(data, to_undirected=True)


In [54]:
nx.write_gexf(G,"graph_data_with.gexf")